In [1]:
print('hola mundo')

hola mundo


importo librerias necesarias para descargar y leer csv 

In [2]:
import pandas as pd
import requests

In [6]:
#codigo clases
url = 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ente-de-turismo/oferta-establecimientos-gastronomicos/oferta_gastronomica.csv'

response = requests.get(url)

# Verifica si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Guarda el contenido de la respuesta en un archivo
    with open('oferta_gastronomica.csv', 'wb') as f:
        f.write(response.content)
    print('Descarga exitosa')
else:
    print('Error al descargar:', response.status_code)

Descarga exitosa


Manera fácil.utilicé el enlace directo del csv de la página de manera contraria me descargaba html

In [7]:
oferta_gastronomica = pd.read_csv('oferta_gastronomica.csv')
oferta_gastronomica.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   long                     2823 non-null   float64
 1   lat                      2823 non-null   float64
 2   id                       2823 non-null   int64  
 3   nombre                   2823 non-null   object 
 4   categoria                2707 non-null   object 
 5   cocina                   2464 non-null   object 
 6   ambientacion             2264 non-null   object 
 7   telefono                 2303 non-null   object 
 8   mail                     472 non-null    object 
 9   horario                  725 non-null    object 
 10  calle_nombre             2823 non-null   object 
 11  calle_altura             2823 non-null   int64  
 12  calle_cruce              6 non-null      object 
 13  direccion_completa       2822 non-null   object 
 14  barrio                  

In [8]:
oferta_gastronomica.head()

,long,lat,id,nombre,categoria,cocina,ambientacion,telefono,mail,horario,calle_nombre,calle_altura,calle_cruce,direccion_completa,barrio,comuna,codigo_postal,codigo_postal_argentino
0,-58.363307,-34.622963,1602,GUANTANAMERA,RESTAURANTE,CUBANA,MODERNO,4314 0770,NaN,NaN,MOREAU DE JUSTO ALICIA,2092,NaN,"MOREAU DE JUSTO, ALICIA AV. 2092",Puerto Madero,Comuna 1,1107.0,C1107AFP
1,-58.392330,-34.621554,676,1843 VINACOTECA,VINERIA,NaN,NaN,4779 0806,NaN,lunes a domingo de 8 a 00,HUMBERTO 1º,1843,NaN,HUMBERTO 1º 1843,San Cristobal,Comuna 3,NaN,NaN
2,-58.453542,-34.560311,487,2080 EXPRESS,RESTAURANTE,TRADICIONAL,CLASICO PORTEÑO,4788 4500,NaN,lunes a domingo de 8 a 00,JURAMENTO,2080,NaN,JURAMENTO AV. 2080,Belgrano,Comuna 13,1428.0,C1428DNH
3,-58.475491,-34.677051,2737,22,CAFE,MINUTAS,MODERNO,NaN,NaN,NaN,MURGUIONDO,4188,NaN,MURGUIONDO 4188,Villa Lugano,Comuna 8,1439.0,C1439FTB
4,-58.425437,-34.585695,2,4 EN 2 I,RESTAURANTE,NaN,NaN,4832 9510,NaN,NaN,BORGES JORGE LUIS,2095,NaN,"BORGES, JORGE LUIS 2095",Palermo,Comuna 14,1425.0,C1425FFA


ahora voy a conectar con mysql

In [13]:
import csv
import requests
import pymysql
from io import StringIO#permite pasar a string
import pandas as pd

importante cerrar cursor siempre para evitar que hallan cambios

A trabajar con sql:

1_ Elegimos las columnas que vamos a necesitar:

In [17]:
# URL del CSV
url = 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ente-de-turismo/oferta-establecimientos-gastronomicos/oferta_gastronomica.csv'

# Descargar el CSV
response = requests.get(url)
response.raise_for_status()  # Asegura que la solicitud se realizó correctamente

# Leer el CSV en un DataFrame
data = StringIO(response.text)
df = pd.read_csv(data)

# Seleccionar las columnas deseadas
columns_of_interest = ['nombre', 'categoria', 'direccion_completa', 'barrio', 'comuna']
df_filtered = df[columns_of_interest]

# Mostrar las primeras filas del DataFrame filtrado
print(df_filtered.head())

# Si quieres guardar el resultado en un nuevo archivo CSV
df_filtered.to_csv('OG_filtrado.csv', index=False)


            nombre    categoria                direccion_completa  \
0     GUANTANAMERA  RESTAURANTE  MOREAU DE JUSTO, ALICIA AV. 2092   
1  1843 VINACOTECA      VINERIA                 HUMBERTO 1Âº 1843   
2     2080 EXPRESS  RESTAURANTE                JURAMENTO AV. 2080   
3               22         CAFE                   MURGUIONDO 4188   
4         4 EN 2 I  RESTAURANTE           BORGES, JORGE LUIS 2095   

          barrio     comuna  
0  Puerto Madero   Comuna 1  
1  San Cristobal   Comuna 3  
2       Belgrano  Comuna 13  
3   Villa Lugano   Comuna 8  
4        Palermo  Comuna 14  


Conecto con MySQL

In [ ]:
import requests
import pandas as pd
from io import StringIO
import mysql.connector


In [49]:
conexion = pymysql.connect(
   host='localhost',
   database='gastronomia',
   user='root',
   password='12345'
)
cursor = conexion.cursor()


Cree una tabla con los siguientes campos: id_local, nombre, categoria, direccion, barrio, comuna en MySQL.

In [36]:
#desde py creo la tabla
#Crear la tabla si no existe
create_table_query = """
CREATE TABLE IF NOT EXISTS establecimientos (
    id_local INT AUTO_INCREMENT PRIMARY KEY,
    nombre VARCHAR(255),
    categoria VARCHAR(255),
    direccion VARCHAR(255),
    barrio VARCHAR(255),
    comuna VARCHAR(255)
)
"""
cursor.execute(create_table_query)

0

In [37]:
conexion.commit()

In [38]:
# Insertar los datos en la tabla
insert_query = """
INSERT INTO establecimientos(nombre, categoria, direccion, barrio, comuna)
VALUES (%s, %s, %s, %s, %s)
"""

# Convertir el DataFrame en una lista de tuplas
data_to_insert = df_filtered.values.tolist()

# Insertar los datos
cursor.executemany(insert_query, data_to_insert)

# Confirmar los cambios
conexion.commit()

# Cerrar la conexión
#cursor.close()
#conexion.close()

print("Datos insertados correctamente.")

ProgrammingError: nan can not be used with MySQL

Tengo este error por no haber limpiado los datos debo eliminar los Nan

In [40]:
# Eliminar filas con NaN
df_filtered.dropna(inplace=True)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_10040\3554913382.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.dropna(inplace=True)


Verifico que no hallan Nan

In [41]:
df_filtered.isnull().sum()

nombre                0
categoria             0
direccion_completa    0
barrio                0
comuna                0
dtype: int64

In [42]:
# Insertar los datos en la tabla
insert_query = """
INSERT INTO establecimientos(nombre, categoria, direccion, barrio, comuna)
VALUES (%s, %s, %s, %s, %s)
"""

# Convertir el DataFrame en una lista de tuplas
data_to_insert = df_filtered.values.tolist()

# Insertar los datos
cursor.executemany(insert_query, data_to_insert)

# Confirmar los cambios
conexion.commit()

print("Datos insertados correctamente.")

Datos insertados correctamente.


In [22]:
og = pd.read_csv('OG_filtrado.csv')

In [23]:
og.head(5)

,nombre,categoria,direccion_completa,barrio,comuna
0,GUANTANAMERA,RESTAURANTE,"MOREAU DE JUSTO, ALICIA AV. 2092",Puerto Madero,Comuna 1
1,1843 VINACOTECA,VINERIA,HUMBERTO 1Âº 1843,San Cristobal,Comuna 3
2,2080 EXPRESS,RESTAURANTE,JURAMENTO AV. 2080,Belgrano,Comuna 13
3,22,CAFE,MURGUIONDO 4188,Villa Lugano,Comuna 8
4,4 EN 2 I,RESTAURANTE,"BORGES, JORGE LUIS 2095",Palermo,Comuna 14


Para armar diccionario de datos 

In [24]:
og.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   nombre              2823 non-null   object
 1   categoria           2707 non-null   object
 2   direccion_completa  2822 non-null   object
 3   barrio              2823 non-null   object
 4   comuna              2823 non-null   object
dtypes: object(5)
memory usage: 110.4+ KB


In [25]:
og.describe()

,nombre,categoria,direccion_completa,barrio,comuna
count,2823,2707,2822,2823,2823
unique,2554,8,2745,47,15
top,MC DONALD'S,RESTAURANTE,FLORIDA 787,Palermo,Comuna 1
freq,53,1990,6,467,870


Las queries con py

a) ¿Cuál es el barrio con mayor cantidad de Pubs?

b) Obtener la cantidad de locales por categoría 

c) Obtener la cantidad de restaurantes por comuna

In [43]:
# Consulta a)
query_a = """
SELECT barrio, COUNT(*) as cantidad_pubs
FROM establecimientos
WHERE categoria = 'Pub'
GROUP BY barrio
ORDER BY cantidad_pubs DESC
LIMIT 1
"""

cursor.execute(query_a)
result_a = cursor.fetchone()

print("Barrio con mayor cantidad de Pubs:", result_a[0], "con", result_a[1], "Pubs")


Barrio con mayor cantidad de Pubs: Recoleta con 8 Pubs


In [44]:
 #Consulta b)
query_b = """
SELECT categoria, COUNT(*) as cantidad_locales
FROM establecimientos
GROUP BY categoria
"""

cursor.execute(query_b)
result_b = cursor.fetchall()

print("Cantidad de locales por categoría:")
for row in result_b:
    print(row[0], ":", row[1])

Cantidad de locales por categoría:
RESTAURANTE : 1990
VINERIA : 22
CAFE : 334
BAR : 246
PUB : 53
CONFITERIA : 55
SANDWICHERIA : 4
DELIVERY & TAKE AWAY : 2


In [50]:
# Consulta c)
query_c = """
SELECT comuna, COUNT(*) as cantidad_restaurantes
FROM establecimientos
WHERE categoria = 'Restaurante'
GROUP BY comuna
"""

cursor.execute(query_c)
result_c = cursor.fetchall()

print("Cantidad de restaurantes por comuna:")
for row in result_c:
    print(row[0], ":", row[1])



Cantidad de restaurantes por comuna:
Comuna 1 : 609
Comuna 13 : 108
Comuna 14 : 366
Comuna 2 : 227
Comuna 7 : 64
Comuna 12 : 42
Comuna 10 : 43
Comuna 4 : 95
Comuna 3 : 145
Comuna 5 : 56
Comuna 9 : 52
Comuna 6 : 87
Comuna 15 : 57
Comuna 8 : 6
Comuna 11 : 33


In [51]:
# Cerrar la conexión
cursor.close()
conexion.close()

Ahora hago las queries en Msql